## The notebook here is used for the following:
        
        . for the dualization task, the objective is to test as many loops as possible
        . Hence, we decided to avoid selecting more than one 2-bar loop 
                 from each performance in groove midi dataset
        . To that end, we needed to check how to select the 2-bar loop that best represents the pattern
        . For the selection, we've decided to use some sort of a simple self-similarity measure
            
            the similarity measure: 
            
            (1) get cosine similarity distance between a loop and every other loop 
                (dA_B) Abs. Value of euclidean distance of A and B (L1 norm of distance)
                                        
            
                                loop1    loop2   loop3     Sum
                        loop1     0.     d1_2    d1_3     d1 = \sum{0+d1_2+d1_3}
                        loop2   d2_1       0     d2_3     d2 = \sum{d2_1+0+d2_3}
                        loop3   d3_1     d3_2     0       d3 = \sum{d3_1+d3_2+0}
                              
                              
            (2) sum the distances for each row 
            (3) Assume the loop with the highest sum to be the most similar one
            
            
            Note: Test this on velocity tensors, utiming tensors and also mixed tensor of vel and utime



In [1]:
import pickle
import os, sys
import pandas as pd
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt

from bokeh.io import output_file, show, save
from bokeh.layouts import row,gridplot
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, ColumnDataSource, PrintfTickFormatter, Title
from bokeh.plotting import figure
from bokeh.transform import transform
from bokeh.palettes import Viridis3, Viridis256
from bokeh.embed import file_html
from bokeh.resources import CDN

from bokeh.plotting import show

sys.path.append("../")
from hvo_sequence.hvo_seq import HVO_Sequence
from note_seq.midi_io import midi_file_to_note_sequence

import soundfile as sf
import pretty_midi
import note_seq
import wandb

from utils_groove_dualize import polysyncs_per_loops_using_hvo_sequence

True


In [2]:
wandb.init(project='gmd_most_similar_loop_selection', entity='behzadhaki')

wandb: Currently logged in as: behzadhaki (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.27 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [3]:
def get_all_beat_loops_per_performance(hvo_seqs, midis, metadata):
    
    # Grab just the beats
    beat_df = metadata[metadata["beat_type"] == "beat"]
    
    # get a list of the master_ids for beats
    beat_master_ids = beat_df['master_id'].unique()
    print("There are {} unique performances in the set".format(len(beat_master_ids.tolist())))
    
    segmented_performances_hvos = []
    segmented_performances_hvo_seqs = []
    segmented_performances_midis = []
    
    metadata_indices = []
    # For each master_ids, group all loops together
    for master_id in beat_master_ids:
        # Get indices for the loops from the same performance 
        # (i.e. with same master_id) 
        loops_in_performance = metadata[metadata["master_id"] == master_id]
        
        # Get the ones that are only beat 
        # (i.e. ignore fills)
        
        loop_indices_in_performance = (metadata[metadata["master_id"] == master_id]).index.to_numpy()
        metadata_indices.append(loop_indices_in_performance[0])
        
        hvos_in_performance = [hvo_seqs[i].hvo for i in loop_indices_in_performance]
        hvo_seqs_in_performance = [hvo_seqs[i] for i in loop_indices_in_performance]
        midis_in_performance = [midis[i] for i in loop_indices_in_performance]
        
        segmented_performances_hvos.append(hvos_in_performance)
        segmented_performances_hvo_seqs.append(hvo_seqs_in_performance)
        segmented_performances_midis.append(midis_in_performance)
        
    segmented_performance_metadata = metadata.iloc[metadata_indices].reset_index(drop=True)
    
    return segmented_performances_hvos, segmented_performances_hvo_seqs, segmented_performances_midis, segmented_performance_metadata

def cosine_sim(loop, performance_loops):
    # loop shape:  [32, 27]
    # performance_loops --> list of n elements each have a shape of [32, 27]
    
    a = loop.flatten()
    similarities = np.array([])
    for performance_loop in performance_loops:  
        performance_loop.shape
        b = performance_loop.flatten()
        if b.shape == a.shape: # Ignore shorter loops
            cosine_sim_for_loop = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
            similarities=np.append(similarities, cosine_sim_for_loop)
    return similarities
    
    
def get_self_cosine_similarity(performance_loops):
    self_sim_matrix = np.array([])
    for ix, loop in enumerate(performance_loops):
        if loop.shape[0] == 32:
            sim = cosine_sim(loop, performance_loops)
            if self_sim_matrix.shape[0]>0:
                self_sim_matrix = np.vstack((self_sim_matrix, sim))
            else: 
                self_sim_matrix = sim
    return self_sim_matrix
    

In [4]:
# -------------------------------- TRAIN --------------------------------------------------------

source_path = "../processed_dataset/Processed_On_19_04_2021_at_21_07_hrs"
print(os.path.join(source_path, "GrooveMIDI_processed_train", "hvo_data.obj"))
train_file = open(os.path.join(source_path, "GrooveMIDI_processed_train", "hvo_sequence_data.obj"),'rb')
hvo_seqs_train_set = pickle.load(train_file)
midi_train_files = open(os.path.join(source_path, "GrooveMIDI_processed_train", "midi_data.obj"),'rb')
midi_train_set = pickle.load(midi_train_files)
metadata_train = pd.read_csv(os.path.join(source_path, "GrooveMIDI_processed_train", "metadata.csv"))

features_in_metadata_train = list(metadata_train.columns)

dataset_size = len(hvo_seqs_train_set)

segmented_performances_hvos_train, segmented_performances_hvo_seqs_train, segmented_performances_midis_train, segmented_performance_metadata_train  = get_all_beat_loops_per_performance(
    hvo_seqs_train_set, midi_train_set, metadata_train)

# -------------------------------- TEST ---------------------------------------------------------

print(os.path.join(source_path, "GrooveMIDI_processed_test", "hvo_data.obj"))
test_file = open(os.path.join(source_path, "GrooveMIDI_processed_test", "hvo_sequence_data.obj"),'rb')
hvo_seqs_test_set = pickle.load(test_file)
midi_test_files = open(os.path.join(source_path, "GrooveMIDI_processed_test", "midi_data.obj"),'rb')
midi_test_set = pickle.load(midi_test_files)
metadata_test = pd.read_csv(os.path.join(source_path, "GrooveMIDI_processed_test", "metadata.csv"))

features_in_metadata_test = list(metadata_test.columns)

dataset_size = len(hvo_seqs_test_set)

segmented_performances_hvos_test, segmented_performances_hvo_seqs_test, segmented_performances_midis_test, segmented_performance_metadata_test = get_all_beat_loops_per_performance(
    hvo_seqs_test_set, midi_test_set, metadata_test)


# -------------------------------- VALIDATION --------------------------------------------------------

print(os.path.join(source_path, "GrooveMIDI_processed_validation", "hvo_data.obj"))
validation_file = open(os.path.join(source_path, "GrooveMIDI_processed_validation", "hvo_sequence_data.obj"),'rb')
hvo_seqs_validation_set = pickle.load(validation_file)
midi_validation_files = open(os.path.join(source_path, "GrooveMIDI_processed_validation", "midi_data.obj"),'rb')
midi_validation_set = pickle.load(midi_validation_files)
metadata_validation = pd.read_csv(os.path.join(source_path, "GrooveMIDI_processed_validation", "metadata.csv"))

features_in_metadata_validation = list(metadata_validation.columns)

dataset_size = len(hvo_seqs_validation_set)

segmented_performances_hvos_validation, segmented_performances_hvo_seqs_validation, segmented_performances_midis_validation, segmented_performance_metadata_validation = get_all_beat_loops_per_performance(
    hvo_seqs_validation_set, midi_validation_set, metadata_validation)

../processed_dataset/Processed_On_19_04_2021_at_21_07_hrs/GrooveMIDI_processed_train/hvo_data.obj
There are 359 unique performances in the set
../processed_dataset/Processed_On_19_04_2021_at_21_07_hrs/GrooveMIDI_processed_test/hvo_data.obj
There are 75 unique performances in the set
../processed_dataset/Processed_On_19_04_2021_at_21_07_hrs/GrooveMIDI_processed_validation/hvo_data.obj
There are 46 unique performances in the set


In [5]:
"""print(metadata_train.at[0,'full_midi_filename'], )
loop_audio_path = "misc/Temp1_{}.wav".format(metadata_train.at[0,'full_midi_filename'].replace("/","-"))
segmented_performances_hvo_seqs_train[0][0].save_audio(filename=loop_audio_path, sr=44100,
                                                 sf_path="../hvo_sequence/soundfonts/Standard_Drum_Kit.sf2")
"""

'print(metadata_train.at[0,\'full_midi_filename\'], )\nloop_audio_path = "misc/Temp1_{}.wav".format(metadata_train.at[0,\'full_midi_filename\'].replace("/","-"))\nsegmented_performances_hvo_seqs_train[0][0].save_audio(filename=loop_audio_path, sr=44100,\n                                                 sf_path="../hvo_sequence/soundfonts/Standard_Drum_Kit.sf2")\n'

In [6]:
# Read your data in pandas dataframe


def plot_similarity(data, similarity, performance_metadata, title_prefix = "test", n_notes_per_loops=None, total_syncopation=None):
    
    
    # here the plot :
    
    
    ix = performance_metadata["master_id"].keys()[0]
    
    title = ""
    title = "{}_{}_{}_{}_{}".format(
        title_prefix, 
        performance_metadata['master_id'][ix].replace("/","."),
        performance_metadata['style_primary'][ix].replace("/","."),
        performance_metadata['style_secondary'][ix].replace("/","."),
        performance_metadata['bpm'][ix],
    )
    
    # You can use your own palette here
    # colors = ['#d7191c', '#fdae61', '#ffffbf', '#a6d96a', '#1a9641']

    # I am using 'Viridis256' to map colors with value, change it with 'colors' if you need some specific colors
    mapper = LinearColorMapper(
        palette=Viridis256, low=min(data["value"]), high=max(data["value"]))

    # Define a figure and tools
    TOOLS = "box_select,lasso_select,pan,wheel_zoom,box_zoom,reset,help"
    __p = figure(
        tools=TOOLS,
        plot_width=300,
        plot_height=250,
        title=title,
        #x_range=list(df.AllColumns1.drop_duplicates()),
        #y_range=list(df.AllColumns2.drop_duplicates()),
        toolbar_location="right",
        x_axis_location="below")
    __p.title.text_font_size = '6pt'
    
    

    # Create rectangle for heatmap
    __p.rect(
        x="AllColumns1",
        y="AllColumns2",
        width=1,
        height=1,
        source=ColumnDataSource(data),
        line_color=None,
        fill_color=transform('value', mapper))

    
    # Add legend
    color_bar = ColorBar(
        color_mapper=mapper,
        location=(0, 0),
        ticker=BasicTicker(desired_num_ticks=10))

    __p.add_layout(color_bar, 'right')
    
    
    # --------- Overall Similarity Plots
    
    x = similarity.sum(axis=0)
    
    softmax_x = np.exp(x)/sum(np.exp(x))
    most_sim_loop_index = np.argmax(softmax_x)
    
    labels = [str(index) for index in list(range(softmax_x.size))]

    sums_fig = figure(tools="", toolbar_location=None,
                      y_range=[0,max(softmax_x)], x_range=__p.x_range,
                      plot_width=__p.plot_width, plot_height=int(__p.plot_height/2))
    
    sums_fig.title.text_font_size = '6pt'
    #sums_fig.segment(0, labels, x, labels, line_width=2, line_color="green", )
    sums_fig.line(labels, softmax_x, line_color="red", line_width=2, )
    
    sums_fig.xaxis.axis_label_text_font_size = "6pt"
    
    sums_fig.add_layout(Title(text="Softmax of Total Cross Cos Similarity Scores", text_font_style="italic", text_font_size="7pt"), 'above')
    sums_fig.add_layout(Title(text="Cross Similarity Rankings per 2-bar loop", text_font_size="6pt"), 'above')
    
    
    grid_setup = [[__p], [sums_fig]]
    
    
    if total_syncopation is not None:
        
        labels = [str(index) for index in list(range(len(total_syncopation)))]

        syncopation_fig = figure(tools="", toolbar_location=None,
                             y_range=[0,max(total_syncopation)], x_range=__p.x_range,
                             plot_width=__p.plot_width, plot_height=int(__p.plot_height/2))

        syncopation_fig.title.text_font_size = '6pt'

        syncopation_fig.line(labels, total_syncopation, line_color="green", line_width=2, )

        syncopation_fig.xaxis.axis_label_text_font_size = "6pt"

        syncopation_fig.add_layout(Title(text="Total Syncopation", text_font_style="italic", text_font_size="7pt"), 'above')

    
        # Put together and return
        grid_setup.append([syncopation_fig])
    
    
    # ------ Total Number of Hits per loop
    
    if n_notes_per_loops is not None:
        labels = [str(index) for index in list(range(len(n_notes_per_loops)))]

        n_notes_fig = figure(tools="", toolbar_location=None,
                          y_range=[0,max(n_notes_per_loops)], x_range=__p.x_range,
                          plot_width=__p.plot_width, plot_height=int(__p.plot_height/2))

        n_notes_fig.title.text_font_size = '6pt'

        n_notes_fig.line(labels, n_notes_per_loops, line_color="blue", line_width=2, )

        n_notes_fig.xaxis.axis_label_text_font_size = "6pt"

        n_notes_fig.add_layout(Title(text="Number of Notes in Loop", text_font_style="italic", text_font_size="7pt"), 'above')

        # Put together and return
        #return gridplot([[__p], [sums_fig], [n_notes_fig]]), most_sim_loop_index   
        
        grid_setup.append([n_notes_fig])
        
    return gridplot(grid_setup), most_sim_loop_index    
    
def column_source_dict(similarity):

    df = pd.DataFrame(similarity)
    df.index.name = 'AllColumns1'
    df.columns.name = 'AllColumns2'

    # Prepare data.frame in the right format
    df = df.stack().rename("value").reset_index()

    data = dict()
    data["AllColumns1"] = list(df["AllColumns1"])
    data["AllColumns2"] = list(df["AllColumns2"])
    data["value"] = list(df["value"])
    
    return data

def plot_hvo_similarities(performance_loop_list, performance_metadata, title_prefix="Test_",
                          total_syncopation = None):
    
    
    
    
    h_sim = get_self_cosine_similarity([x[:,:9] for x in performance_loop_list])
    
    # Calculate note counts per loop
    hits_per_loops = [x[:,:9] for x in performance_loop_list]
    note_count_per_loops = []
    for hits_in_loop in hits_per_loops:
        vals, counts = np.unique(hits_in_loop.flatten(), return_counts=True)
        count_dict = dict(zip(vals, counts))
        note_count_per_loops.append(count_dict[1])
        
    
    if len(h_sim)==0:
        return None, None
    
    try:
    
        p_h, _ = plot_similarity(column_source_dict(h_sim), h_sim, performance_metadata, title_prefix = title_prefix+"H_", n_notes_per_loops=note_count_per_loops, total_syncopation=total_syncopation)

        v_sim = get_self_cosine_similarity([x[:,9:18] for x in performance_loop_list])
        p_v, _ = plot_similarity(column_source_dict(v_sim), v_sim, performance_metadata, title_prefix = title_prefix+"V_", n_notes_per_loops=note_count_per_loops, total_syncopation=total_syncopation)

        o_sim = get_self_cosine_similarity([x[:,18:] for x in performance_loop_list])
        p_o, _ = plot_similarity(column_source_dict(o_sim), o_sim, performance_metadata, title_prefix = title_prefix+"O_", n_notes_per_loops=note_count_per_loops, total_syncopation=total_syncopation)

        vo_sim = get_self_cosine_similarity([x[:,9:] for x in performance_loop_list])
        p_vo, most_sim_loop_index = plot_similarity(column_source_dict(vo_sim), vo_sim, performance_metadata, title_prefix = title_prefix+"VO_", n_notes_per_loops=note_count_per_loops, total_syncopation=total_syncopation)


        hvo_sim = get_self_cosine_similarity(performance_loop_list)
        p_hvo, _ = plot_similarity(column_source_dict(hvo_sim), hvo_sim, performance_metadata, title_prefix = title_prefix+"HVO_", n_notes_per_loops=note_count_per_loops, total_syncopation=total_syncopation)

        return gridplot([[p_h, p_v, p_o], [p_hvo, p_vo]]), most_sim_loop_index
    except:
    
        return None, None

def midi_to_audio(midi_path, 
                  audio_path="misc/full.wav",
                  sr=44100,
                  sf_path="../hvo_sequence/soundfonts/Standard_Drum_Kit.sf2"):
        pm = pretty_midi.PrettyMIDI(midi_path)
        audio = pm.fluidsynth(sf2_path=sf_path, fs=sr)
        sf.write(audio_path, audio, sr, 'PCM_24')
        return audio
    
    

In [7]:
def find_most_similar_and_send_to_wandb(
    segmented_performances_hvos, segmented_performances_hvo_seqs, segmented_performances_midis,
    metadata, 
    output_ROOT_PATH="misc/", 
    filename_prefix="Test Train Evaluation", 
    full_performance_midi_path="../resources/source_dataset/groove/",
    sf_path="../hvo_sequence/soundfonts/Standard_Drum_Kit.sf2", 
    epoch_count=0):
    
    for ix in range(len(segmented_performances_hvos)):
        
        try:
            total_sync = polysyncs_per_loops_using_hvo_sequence(segmented_performances_hvo_seqs[ix])
        except:
            total_sync = None
        print("ix", ix)
        
        if len(segmented_performances_hvos[ix])>1:
            
            
            fig, most_sim_loop_index = plot_hvo_similarities(
                segmented_performances_hvos[ix], 
                performance_metadata=metadata[metadata.index==ix], 
                title_prefix=filename_prefix,
                total_syncopation = total_sync)

            #show(fig)
            if fig is not None:
                filename_formatless = os.path.join(output_ROOT_PATH, filename_prefix+
                                        metadata.at[ix,"full_midi_filename"].replace("/","-")).split(".")[0]

                # Send html 
                html_filename = filename_formatless+".html"
                save(fig, html_filename)

                # save full performance audio
                midi_path = os.path.join(full_performance_midi_path, metadata.at[ix,"full_midi_filename"])
                #ns = midi_file_to_note_sequence(midi_path)
                performance_audio_path = filename_formatless+"_best_2bar_full.wav"
                audio = midi_to_audio(midi_path, audio_path=performance_audio_path)

                # save most representative 2Bar performance audio 
                hvo_seq = segmented_performances_hvo_seqs[ix][most_sim_loop_index]
                loop_audio_path = filename_formatless+"_best_2bar_segment_{}.wav".format(most_sim_loop_index)
                hvo_seq.save_audio(filename=loop_audio_path, sr=44100,
                           sf_path=sf_path)

                # save midi
                midi_loop_path = filename_formatless+"_best_2bar_segment_{}.mid".format(most_sim_loop_index)
                note_sequence = note_seq.midi_to_note_sequence(segmented_performances_midis[ix][most_sim_loop_index])
                note_seq.midi_io.note_sequence_to_midi_file(note_sequence, midi_loop_path)

                # Log data
                wandb.log(
                    {
                        'epoch': epoch_count,
                        "Audio": 
                        [
                            wandb.Audio(performance_audio_path, 
                                        caption=performance_audio_path.split("/")[-1]),
                            wandb.Audio(loop_audio_path, 
                                        caption=loop_audio_path.split("/")[-1])
                        ],

                        "Similarity_Plots": [wandb.Html(open(html_filename))]
                    })


                epoch_count = epoch_count+1
    
    return epoch_count
    
    

In [8]:
epoch_count = find_most_similar_and_send_to_wandb(
    segmented_performances_hvos_train,
    segmented_performances_hvo_seqs_train,
    segmented_performances_midis_train,
    segmented_performance_metadata_train,
    output_ROOT_PATH="misc/", 
    filename_prefix="Train/", 
    full_performance_midi_path="../resources/source_dataset/groove/",
    sf_path="../hvo_sequence/soundfonts/Standard_Drum_Kit.sf2",
    epoch_count=0)


epoch_count = find_most_similar_and_send_to_wandb(
    segmented_performances_hvos_test, 
    segmented_performances_hvo_seqs_test, 
    segmented_performances_midis_test,
    segmented_performance_metadata_test,
    output_ROOT_PATH="misc/", 
    filename_prefix="Test/", 
    full_performance_midi_path="../resources/source_dataset/groove/",
    sf_path="../hvo_sequence/soundfonts/Standard_Drum_Kit.sf2",
    epoch_count=epoch_count)

find_most_similar_and_send_to_wandb(
    segmented_performances_hvos_validation, 
    segmented_performances_hvo_seqs_validation, 
    segmented_performances_midis_validation,
    segmented_performance_metadata_validation,
    output_ROOT_PATH="misc/", 
    filename_prefix="Validation/", 
    full_performance_midi_path="../resources/source_dataset/groove/",
    sf_path="../hvo_sequence/soundfonts/Standard_Drum_Kit.sf2",
    epoch_count=epoch_count)


ix 0


/Users/behzadhaki/.local/lib/python3.6/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/Users/behzadhaki/.local/lib/python3.6/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


ix 1


/Users/behzadhaki/.local/lib/python3.6/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/Users/behzadhaki/.local/lib/python3.6/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


ix 2


...

ix 44


/Users/behzadhaki/.local/lib/python3.6/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/Users/behzadhaki/.local/lib/python3.6/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


ix 45


/Users/behzadhaki/.local/lib/python3.6/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/Users/behzadhaki/.local/lib/python3.6/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


388

In [9]:
len(segmented_performances_hvo_seqs_train)
len(segmented_performances_midis_train[72])
len(segmented_performances_hvos_train[93])
len(segmented_performances_hvos_train[92][2])

segmented_performance_metadata_train[segmented_performance_metadata_train.index==92]

,Unnamed: 0,drummer,session,loop_id,master_id,style_primary,style_secondary,bpm,beat_type,time_signature,full_midi_filename,full_audio_filename
92,6781,drummer3,session1,drummer3/session1/23:000,drummer3/session1/23,jazz,None,120,beat,4-4,drummer3/session1/23_jazz_120_beat_4-4.mid,drummer3/session1/23_jazz_120_beat_4-4.wav


In [10]:
segmented_performances_hvos_train[0]

[array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.  

In [11]:
segmented_performances_hvo_seqs[0]

NameError: name 'segmented_performances_hvo_seqs' is not defined

In [ ]:
source_path = "../processed_dataset/Processed_On_19_04_2021_at_21_07_hrs"

metadata_train = pd.read_csv(os.path.join(source_path, "GrooveMIDI_processed_train", "metadata.csv"))

In [ ]:
hits_per_loops = [x[:,:9] for x in segmented_performances_hvos_train[0]]
note_count_per_loops = []
for hits_in_loop in hits_per_loops:
    vals, counts = np.unique(hits_in_loop.flatten(), return_counts=True)
    count_dict = dict(zip(vals, counts))
    note_count_per_loops.append(count_dict[1])
print("note_count_per_loops", note_count_per_loops)

In [ ]:
np.unique(hits_in_loop.flatten(), return_counts=True)

In [ ]:
a = [[1], [2, 3]]

In [ ]:
a.append([5])

In [ ]:
a
